## Gaussian Kernel
高斯滤波是一种线性滤波，图像上的每一个像素点的值，都由其本身和邻域内其他像素点的值经过加权平均后得到。由高斯核扫描图像中每一个像素点，将邻域内各个像素值与对应位置的权值相称并求和。从数学的角度来看，高斯滤波的过程是图像与高斯正态分布做卷积操作。 

In [1]:
import cv2
import numpy as np

In [2]:
def esc():
    key = cv2.waitKey()
    if key == 27:
        cv2.destroyAllWindows()

In [13]:
# img = cv2.imread("/home/ubuntu/Downloads/Learn_CV/cv_lesson02/img/scarlett_4.jpeg")
# img = cv2.imread("/home/ubuntu/Downloads/Learn_CV/cv_lesson02/img/000001.jpg")
img = cv2.imread("/home/ubuntu/Downloads/Learn_CV/cv_lesson02/img/scarlett_3.jpeg")
cv2.imshow('scarlett', img)
esc()
        

---
cv2.**GaussianBlur**(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]) → dst  
**Parameters:**	 
  - **src:** - input image; the image can have any number of channels, which are processed independently, but the depth should be CV_8U, CV_16U, CV_16S, CV_32F or CV_64F.  

  - **dst:** - output image of the same size and type as src. 

  - **ksize:** - Gaussian kernel size. ksize.width and ksize.height can differ but they both must be **positive and odd**. Or, they can be zero’s and then they are computed from sigma* . 

  - **sigmaX:** - Gaussian kernel standard deviation in X direction.  

  - **sigmaY:** - Gaussian kernel standard deviation in Y direction; if sigmaY is zero, it is set to be equal to sigmaX, if both sigmas are zeros, they are computed from ksize.width and ksize.height , respectively (see getGaussianKernel() for details); to fully control the result regardless of possible future modifications of all this semantics, it is recommended to specify all of ksize, sigmaX, and sigmaY.  

  - **borderType:** - pixel extrapolation method (see borderInterpolate() for details).  

In [4]:
# 高斯分布的方差（variance，第三个参数）控制图像的模糊程度，越大越模糊
g_img = cv2.GaussianBlur(img, (7, 7), 1)
cv2.imshow('gaussian_blur_scarlett', g_img)
cv2.imshow('scarlett', img)
esc()

---
cv2.**getGaussianKernel**(ksize, sigma[, ktype]) → retval
**Parameters:**	
- **ksize** – Aperture size. It should be odd ( \texttt{ksize} \mod 2 = 1 ) and positive.
- **sigma** – Gaussian standard deviation. If it is non-positive, it is computed from ksize as sigma = 0.3*((ksize-1)*0.5 - 1) + 0.8 .
- **ktype** – Type of filter coefficients. It can be CV_32f or CV_64F .  
The function computes and returns the $kesize * 1$ matrix of Gaussian filter coefficients:
$$G = \alpha * e^-\frac{(i-(ksize-1)/2)^2}{(2×sigma^2)}$$
where i=0..$kesize-1$ and $\alpha$ is the scale factor chosen so that $\sum_i G_i=1$.


In [5]:
kernel = cv2.getGaussianKernel(7, 5)
print(kernel)

[[ 0.12895603]
 [ 0.14251846]
 [ 0.15133131]
 [ 0.1543884 ]
 [ 0.15133131]
 [ 0.14251846]
 [ 0.12895603]]


---
**sepFilter2D**
**ddepth:** Destination image depth. The following combination of src.depth() and ddepth are supported:
- src.depth() = CV_8U, ddepth = -1/CV_16S/CV_32F/CV_64F
- src.depth() = CV_16U/CV_16S, ddepth = -1/CV_32F/CV_64F
- src.depth() = CV_32F, ddepth = -1/CV_32F/CV_64F
- src.depth() = CV_64F, ddepth = -1/CV_64F

The function applies a separable linear filter to the image. That is, first, every row of src is filtered with the 1D kernel kernelX . Then, every column of the result is filtered with the 1D kernel kernelY . The final result shifted by delta is stored in dst .

#### 二维高斯分离特性：
$$G(x,y) = \frac{1}{2\pi\sigma^2}e^-\frac{x^2+y^2}{2\sigma^2} = G(x)*G(y)$$

In [6]:
# 对比一维、二维高斯核运行效果
# 二维时间复杂度：m*n*ksize*ksize
# 一维时间复杂度：m*n*ksize*2
# 一维更快
g1_img = cv2.GaussianBlur(img, (7, 7), 6)
g2_img = cv2.sepFilter2D(img, -1, kernel, kernel)
cv2.imshow('scarlett_1', g1_img)
cv2.imshow('scarlett_2', g2_img)
esc()

****
## Other Applications
### 2nd derivative: laplacian(双边缘效果)
一阶导： $f(x+) - f(x)$  
二阶导： $f(x+2)- 2f(x+1) + f(x)$
一阶导粗边；二阶导精细结构，双边效果
- 图像： 5  5  4  3  2  1  0  0  0  6  0 0 0
- 一阶： 0 -1 -1 -1 -1 -1  0  0  6 -6  0 0 0
- 二阶：-1  0  0  0  0  0  1  0  6 -12 6 0 0 


In [7]:
kernel_lap = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]], np.float32)
lap_img = cv2.filter2D(img, -1, kernel = kernel_lap)
cv2.imshow('lap_scarlett', lap_img)
esc()

### 应用： 图像锐化 = edge+ori
>app: sharpen
图像+edge=更锐利地图像，因为突出边缘

In [8]:
kernel_sharp = np.array([[0, 1, 0], [1, -3, 1], [0, 1, 0]], np.float32)
sha_img = cv2.filter2D(img, -1, kernel=kernel_sharp)
cv2.imshow('lap_scarlett', lap_img)
cv2.imshow('sha_scarlett', sha_img)
esc()

---
- 这样不对，因为，周围有4个1，中间是-3，虽然有边缘效果，但是周围得1会使得原kernel有滤波效果，使图像模糊；
- 解决：所以取kernel_lap得相反数，再加上原图像，这样突出了中心像素，效果类似于小方差的高斯，所以可以既有边缘效果，又保留图像清晰度

In [9]:
kernel_sharp2 = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]], np.float32) 
sha_img2 = cv2.filter2D(img, -1, kernel=kernel_sharp2)
kernel_sharp3 = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], np.float32) 
sha_img3 = cv2.filter2D(img, -1, kernel=kernel_sharp3)
cv2.imshow('sha_scarlett2', sha_img2)
cv2.imshow('lap_scarlett', lap_img)
cv2.imshow('sha_scarlett', sha_img)
cv2.imshow('sha_scarlett3', sha_img3)
esc()

---
更“凶猛”的边缘效果
不仅考虑x-y方向上的梯度，同时考虑了对角线方向上的梯度

In [ ]:
kernel_sharp4 = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], np.float32)
sha_img4 = cv2.filter2D(img, -1, kernel=kernel_sharp4)
cv2.imshow('sha_scarlett2', sha_img4)
esc()

## Edge Detection
#### x axis
x方向梯度，检测y方向的边

In [14]:
edgex = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], np.float32)
sharp_img = cv2.filter2D(img, -1, kernel=edgex)
cv2.imshow('edgex_scarlett', sharp_img)
esc()

#### y axis
y方向梯度，检测x方向的边 

In [15]:
edgey = np.array([[-1, 0, -1], [-2, 0, 2], [-1, 0, 1]], np.float32)
sharpy_img = cv2.filter2D(img, -1, kernel=edgey)
cv2.imshow('edgex_scarlett', sharp_img)
cv2.imshow('edgey_scarlett', sharpy_img)
esc()

## 角点
cv.**CornerHarris**(image, harris_dst, blockSize, aperture_size=3, k=0.04) → None  
- src – Input single-channel 8-bit or floating-point image.
- dst – Image to store the Harris detector responses. It has the type CV_32FC1 and the same size as src .
- **blockSize** – Neighborhood size (see the details on cornerEigenValsAndVecs() ).
- **ksize** – Aperture parameter for the Sobel() operator.
- **k** – Harris detector free parameter. See the formula below.
- borderType – Pixel extrapolation method. See borderInterpolate() .

In [3]:
img = cv2.imread("/home/ubuntu/Downloads/Learn_CV/cv_lesson02/img/building.jpeg")
img = cv2.resize(img, (600, 500))
img_gray = np.float32(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
img_harris = cv2.cornerHarris(img_gray, 2, 3, 0.05)
cv2.imshow('img_harris', img_harris)
esc()

#### 看不清
没法看原因：1. float类型； 2. img_harris本质上是每个pixel对于Harris函数的响应值
没有看的价值

In [5]:
thres = 0.05 * np.max(img_harris)
img[img_harris > thres] = [0, 0, 255]
cv2.imshow('building_harris', img)
esc()

#### 为了看清
膨胀

In [8]:
img_harris = cv2.dilate(img_harris, None)
thres = 0.05 * np.max(img_harris)
img[img_harris > thres] = [0, 0, 255]
cv2.imshow('building_harris', img)
esc()

# SIFT
cv2.xfeatures2d.SIFT_create()报错解决：[click](https://stackoverflow.com/questions/52305578/sift-cv2-xfeatures2d-sift-create-not-working-even-though-have-contrib-instal)

In [10]:
img = cv2.imread("/home/ubuntu/Downloads/Learn_CV/cv_lesson02/img/scarlett_4.jpeg")
print(img.shape)
# create sift class
sift = cv2.xfeatures2d.SIFT_create()
# detect SIFT
kp = sift.detect(img, None) # None for mask
# compute SIFT descritor
kp, des = sift.compute(img, kp)
print(kp[0].pt)
print(kp[0].size)
print(kp[0].angle)
print(kp[0].response)
print(kp[0].octave)
print(kp[0].class_id)
print(type(kp[0]))
print(des.shape)

(612, 570, 3)
(4.606494426727295, 534.2283935546875)
2.0018608570098877
0.555877685546875
0.024253377690911293
7864831
-1
<class 'cv2.KeyPoint'>
(680, 128)


In [4]:
img_sift = cv2.drawKeypoints(img, kp, outImage=np.array([]), 
                             flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow('scarlett_sift', img_sift)
esc()